## 第十二章 pandas 高级应用

### 12.1 分类数据

In [2]:
import numpy as np
import pandas as pd

values = pd.Series(['apple', 'orange', 'apple', 'apple'] *2)
values

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [3]:
pd.unique(values)

array(['apple', 'orange'], dtype=object)

In [4]:
pd.value_counts(values)

apple     6
orange    2
dtype: int64

In [5]:
values = pd.Series([0, 1, 0, 0] * 2)
dim = pd.Series(['apple', 'orange'])
dim

0     apple
1    orange
dtype: object

In [6]:
dim.take(values)     # 使用 task()方法存储原始的字符串 Series, 分类编码

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

In [7]:
# pandas 的分类类型
fruits = ['apple', 'orange', 'apple', 'apple'] * 2
N = len(fruits)
df = pd.DataFrame({'fruit': fruits,
                  'basket_id': np.arange(N),
                  'count': np.random.randint(3, 15, size=N),
                  'weight': np.random.uniform(0, 4, size=N)},
                  columns=['basket_id', 'fruit', 'count', 'weight'])
df

,basket_id,fruit,count,weight
0,0,apple,13,1.168792
1,1,orange,11,0.384080
2,2,apple,6,2.761167
3,3,apple,10,3.635060
4,4,apple,12,3.727640
5,5,orange,3,3.227282
6,6,apple,13,0.573331
7,7,apple,4,3.608193


In [8]:
fruit_cat = df['fruit'].astype('category')
fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

In [9]:
fruit_cat.values

[apple, orange, apple, apple, apple, orange, apple, apple]
Categories (2, object): [apple, orange]

In [10]:
type(fruit_cat.values)

pandas.core.arrays.categorical.Categorical

In [13]:
# 分组计算
np.random.seed(12345)  # ?

In [14]:
draws = np.random.randn(1000)
draws[:5]

array([-0.20470766,  0.47894334, -0.51943872, -0.5557303 ,  1.96578057])

In [15]:
bins = pd.cut(draws, 4)
bins

[(-1.23, 0.489], (-1.23, 0.489], (-1.23, 0.489], (-1.23, 0.489], (0.489, 2.208], ..., (-1.23, 0.489], (-1.23, 0.489], (-1.23, 0.489], (0.489, 2.208], (0.489, 2.208]]
Length: 1000
Categories (4, interval[float64]): [(-2.956, -1.23] < (-1.23, 0.489] < (0.489, 2.208] < (2.208, 3.928]]

In [18]:
bins = pd.qcut(draws, 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
bins

[Q2, Q3, Q2, Q2, Q4, ..., Q3, Q2, Q1, Q3, Q4]
Length: 1000
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

In [19]:
bins.codes[:10]

array([1, 2, 1, 1, 3, 3, 2, 2, 3, 3], dtype=int8)

In [21]:
bins = pd.Series(bins, name='quartile')
results = (pd.Series(draws).groupby(bins).agg(['count', 'min', 'max']).reset_index())
results

,quartile,count,min,max
0,Q1,250,-2.949343,-0.685484
1,Q2,250,-0.683066,-0.010115
2,Q3,250,-0.010032,0.628894
3,Q4,250,0.634238,3.927528


In [22]:
results['quartile']

0    Q1
1    Q2
2    Q3
3    Q4
Name: quartile, dtype: category
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

In [ ]:
# 2.分组 -- qcut & cut
qcut = pd.qcut(p_change, 10)   # qcut： 分成10组（注意：643数量等分）
qcut.value_counts()            # value_counts()： 统计每组的个数

In [ ]:
# 自定义分组区间  --  pd.cut()
bins = [-100, -7, -5, -3, 0, 3, 5, 7, 100]
cut_data = pd.cut(p_change, bins)
cut_data.value_counts()

In [ ]:
# 3.one-hot编码(转换成哑变量矩阵) -- pd.dummies()
dummies = pd.get_dummies(cut_data, prefix="rise")    # prefix: 分组名字
dummies.head()

In [23]:
# groupby 分组&聚合
# 准备数据
col =pd.DataFrame({'color': ['white','red','green','red','green'], 'object': ['pen','pencil','pencil','ashtray','pen'],'price1':[5.56,4.20,1.30,0.56,2.75],'price2':[4.75,4.12,1.60,0.75,3.15]})
col

,color,object,price1,price2
0,white,pen,5.56,4.75
1,red,pencil,4.20,4.12
2,green,pencil,1.30,1.60
3,red,ashtray,0.56,0.75
4,green,pen,2.75,3.15


In [25]:
# 根据颜色分组，求分组price1的平均值
# col["price1"].groupby(col["color"]).mean()   # Series 格式
col.groupby(["color"])["price1"].mean()        # DataFrame 格式 -- 推荐

color
green    2.025
red      2.380
white    5.560
Name: price1, dtype: float64

In [26]:
col.groupby(["color"], as_index=False)["price1"].mean()   # as_index=False

,color,price1
0,green,2.025
1,red,2.380
2,white,5.560
